In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
spark = SparkSession.builder.appName("lab_2_spark").getOrCreate()
sc = spark.sparkContext

In [3]:
perimetre = spark.read.csv("data_clients/sample_perimetre.csv", header=True)
histo_client_raw = spark.read.csv("data_clients/sample_histo_client.csv", header=True)
histo_train_raw = spark.read.csv("data_clients/sample_histo_train.csv", header=True)
histo_lowcost_raw = spark.read.csv("data_clients/sample_histo_lowcost.csv", header=True)
visites_raw = spark.read.csv("data_clients/sample_visites.csv", header=True)

# Column conversion to double

In [4]:
def cast_columns_of_df(df, cols_to_cast, col_to_keep, cast_type="double"):
    return df.select(col_to_keep + [(df[feature].cast(cast_type)) 
                                    for feature in cols_to_cast if "ID_CLIENT" not in feature])


client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']

In [5]:
col_to_keep = "ID_CLIENT"

histo_client_raw = cast_columns_of_df(histo_client_raw, histo_client_raw.columns,[col_to_keep],)
histo_train_raw = cast_columns_of_df(histo_train_raw, histo_train_raw.columns, [col_to_keep])
histo_lowcost_raw = cast_columns_of_df(histo_lowcost_raw, histo_lowcost_raw.columns, [col_to_keep])
visites_raw = cast_columns_of_df(visites_raw, visites_raw.columns, [col_to_keep])

## Joining

In [6]:
lowcost = perimetre.join(histo_client_raw, how="left", on=col_to_keep).join(histo_train_raw, how="left", on=col_to_keep).join(histo_lowcost_raw, how="left", on=col_to_keep).join(visites_raw, how="left", on=col_to_keep)

### Checking same number of users

In [7]:
lowcost.count() == perimetre.count()

True

### Save data to disk

In [ ]:
#lowcost.write.save("data_clients/lowcost.csv",format="csv")

In [8]:
# Number of columns
list(enumerate(lowcost.columns))

[(0, 'ID_CLIENT'),
 (1, 'anciennete'),
 (2, 'recence_cmd'),
 (3, 'AGE'),
 (4, 'LBL_STATUT_CLT'),
 (5, 'LBL_GEO_AIR'),
 (6, 'LBL_GRP_SEGMENT_NL'),
 (7, 'LBL_SEG_COMPORTEMENTAL'),
 (8, 'LBL_GEO_TRAIN'),
 (9, 'LBL_SEGMENT_ANTICIPATION'),
 (10, 'FLG_CMD_CARTE_1225'),
 (11, 'nb_od'),
 (12, 'mean_nb_passagers'),
 (13, 'mean_duree_voyage'),
 (14, 'mean_mt_voyage'),
 (15, 'mean_tarif_loisir'),
 (16, 'mean_classe_1'),
 (17, 'mean_pointe'),
 (18, 'mean_depart_we'),
 (19, 'flg_cmd_lowcost'),
 (20, 'flg_track_nl_lowcost'),
 (21, 'flg_track_nl'),
 (22, 'days_since_last_visit'),
 (23, 'tx_conversion')]

In [9]:
# Distinct values
lowcost.select("LBL_STATUT_CLT").distinct().show()

+--------------+
|LBL_STATUT_CLT|
+--------------+
|          null|
+--------------+



In [142]:
continue_columns = lowcost.columns[1:]
lowcost.describe()

DataFrame[summary: string, ID_CLIENT: string, anciennete: string, recence_cmd: string, AGE: string, LBL_STATUT_CLT: string, LBL_GEO_AIR: string, LBL_GRP_SEGMENT_NL: string, LBL_SEG_COMPORTEMENTAL: string, LBL_GEO_TRAIN: string, LBL_SEGMENT_ANTICIPATION: string, FLG_CMD_CARTE_1225: string, nb_od: string, mean_nb_passagers: string, mean_duree_voyage: string, mean_mt_voyage: string, mean_tarif_loisir: string, mean_classe_1: string, mean_pointe: string, mean_depart_we: string, flg_cmd_lowcost: string, flg_track_nl_lowcost: string, flg_track_nl: string, days_since_last_visit: string, tx_conversion: string]

# NaN handling

In [30]:
continue_columns = lowcost.columns[1:]
df = lowcost
dm = [df.select(f.mean(feature)).collect()[0][0] for feature in continue_columns]

In [33]:
df_nf = df.select([f.when(df[feature].isNotNull(),df[feature])\
           .otherwise(-1).alias(feature) for i,feature in enumerate([col_to_keep])]\
          +[f.when(df[feature].isNotNull(),df[feature])\
            .otherwise(dm[i]).alias(feature) for i,feature in enumerate(continue_columns)]) # nf nan-free

In [10]:
def input_df(df):
    ds = df.select('ID_CLIENT',
    f.when(df.LBL_GEO_TRAIN.isin(['Toulouse', 'Lille', 'Dijon',
                                  'Lyon', 'Marseille', 'Paris',
                                  'Nice', 'Limoges','Rouen','Rennes',
                                  'Montpellier', 'Bordeaux', 'Metz',
                                  'Strasbourg']), df.LBL_GEO_TRAIN)\
               .otherwise('na').alias('geo_train'),
    f.when(df.LBL_GEO_AIR.isin(['Aéroports de Paris Orly',
                                'Aéroport de Bâle-Mulhouse / Bassel',
                                'Aéroport Lille Lesquin', 'Aéroport de Rennes',
                                'Aéroport de Nantes Atlantique',
                                'Aéroport de Marseille Provence  (MRS)', 
                                'Aéroport de Bordeaux Mérignac',
                                'Aéroports de Paris Roissy-Charles-de Gaulle', 
                                "Aéroport de Nice Côte d'Azur",
                                'Aéroport de Strasbourg',
                                'Aéroport de Lyon - Saint Exupéry', 
                                'Aéroport de Toulouse Blagnac']), df.LBL_GEO_AIR)\
               .otherwise('na').alias('geo_air'),
    f.when(df.FLG_CMD_CARTE_1225 == '1', '1')\
                   .otherwise('0').alias('cc_jeunes'),
    f.when(df.LBL_STATUT_CLT.isin(['Tres grand', 'Nouveau actif',
                                   'Moyen moins', ' Prospect', ' Petit',
                                   'Inactif', 'Tres petit',
                                   'Nouveau prospect', 'Moyen plus',
                                   'Grand']), df.LBL_STATUT_CLT)\
                   .otherwise('na').alias('segt_rfm'),
    f.when(df.LBL_SEGMENT_ANTICIPATION.isin(['Peu Anticipateur', 'Tres Anticipateur',
                                             'Anticipateur', 'Mixte', 'Non Anticipateur',
                                             'Non Defini']), df.LBL_SEGMENT_ANTICIPATION)\
                   .otherwise('na').alias('segt_anticipation'),
    f.when(df.LBL_SEG_COMPORTEMENTAL.isin(['Mono-commande',
                                           'Comportement Pro',
                                           'Exclusifs Agence', 
                                           'Anticipateurs Methodiques',
                                           'Chasseurs Bons Plans', 
                                           'Rythmes scolaires', 'Nouveaux',
                                           'Sans contraintes']),
           df.LBL_SEG_COMPORTEMENTAL).otherwise('na').alias('segt_comportemental'), 
    f.when(df.LBL_GRP_SEGMENT_NL.isin(['Endormi', 'Spectateur', 'Acteur',
                                       'Eteint', 'Non defini']),
           df.LBL_GRP_SEGMENT_NL).otherwise('na').alias('segt_nl'),
    f.when(((df.AGE > 0) & (df.AGE < 100)), df.AGE)\
                   .otherwise(-1).alias('age'),
    f.when(df.recence_cmd >= 0, df.recence_cmd)\
                   .otherwise(-1).alias('recence_cmd'),
    f.when(((df.mean_duree_voyage > 0) & (df.mean_duree_voyage < 750)),
           df.mean_duree_voyage).otherwise(-1).alias('mean_duree_voyage'),
    f.when(df.days_since_last_visit >= 0, df.days_since_last_visit)\
                   .otherwise(-1).alias('recence_visite'),
    f.when(df.mean_mt_voyage > 0, df.mean_mt_voyage)\
                   .otherwise(-1).alias('mean_mt_voyage'),
    f.when(df.anciennete >= 0, df.anciennete)\
                   .otherwise(-1).alias('anciennete'),
    f.when(df.nb_od > 0, df.nb_od)\
                   .otherwise(-1).alias('nb_od'),
    f.when(df.mean_nb_passagers > 0, df.mean_nb_passagers)\
                   .otherwise(-1).alias('mean_nb_passagers'),
    f.when(df.mean_tarif_loisir >= 0, df.mean_tarif_loisir)\
                   .otherwise(-1).alias('mean_tarif_loisir'),
    f.when(df.mean_classe_1 >= 0, df.mean_classe_1)\
                   .otherwise(-1).alias('mean_classe_1'),
    f.when(df.mean_pointe >= 0, df.mean_pointe)\
                   .otherwise(-1).alias('mean_pointe'),
    f.when(df.mean_depart_we >= 0, df.mean_depart_we)\
                   .otherwise(-1).alias('mean_depart_we'),
    f.when(df.tx_conversion >= 0, df.tx_conversion)\
                   .otherwise(-1).alias('tx_conversion'),
    f.when(df.flg_cmd_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_cmd_lowcost'),
    f.when(df.flg_track_nl_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_track_nl_lowcost'), 
    f.when(df.flg_track_nl == 1, '1')\
                   .otherwise('0').alias('flg_track_nl'))
    
    return ds

In [37]:
#df_ = input_df(lowcost)
df_ = input_df(df_nf)

In [43]:
y = df_["flg_cmd_lowcost"]
X = df_.drop("flg_cmd_lowcost")

In [36]:
df_.toPandas().head(3)

,ID_CLIENT,geo_train,geo_air,cc_jeunes,segt_rfm,segt_anticipation,segt_comportemental,segt_nl,age,recence_cmd,...,nb_od,mean_nb_passagers,mean_tarif_loisir,mean_classe_1,mean_pointe,mean_depart_we,tx_conversion,flg_cmd_lowcost,flg_track_nl_lowcost,flg_track_nl
0,000843db32fbaecfbb047ca0bb04b1f9f4d9425a,na,na,0,na,na,na,na,36.772698,36.0,...,1.0,1.0,0.0,0.0,0.00,0.00,0.111111,1,0,0
1,001338752ea32d9de129c8f8bdf3e2224cf0bd71,na,na,0,na,na,na,na,35.000000,25.0,...,1.0,1.0,1.0,1.0,0.00,0.00,0.130435,1,0,0
2,003fb9dca8de374386d0fa97b570950583111931,na,na,1,na,na,na,na,25.000000,15.0,...,3.0,1.5,0.5,0.0,0.25,0.25,1.000000,1,0,1


# MLib

In [41]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression

##### features engineering et modélisation

In [44]:

def preprocessed_df(df, label="flg_cmd_lowcostIndex"):
    max_values_to_define_str_cols = 10
    id_col = 'ID_CLIENT'
    
    dty = dict(df.dtypes)
    str_cols = [k for k, v in dty.items() if v == 'string']
    str_cols.remove(id_col)
    
    for c in str_cols:
        stringIndexer = StringIndexer(inputCol=c, outputCol=c+"Index")
        model_str = stringIndexer.fit(df)
        df = model_str.transform(df).drop(c)

    input_cols = df.columns
    input_cols.remove(id_col)
    input_cols.remove(label)
    
    assembler = VectorAssembler(inputCols=input_cols,
                            outputCol="features")
    df = assembler.transform(df)
    
    featureIndexer = VectorIndexer(inputCol="features", 
                   outputCol="indexedFeatures", 
                   maxCategories=max_values_to_define_str_cols).fit(df)
    return featureIndexer.transform(df), df


data, dff = preprocessed_df(df_)

Py4JJavaError: An error occurred while calling o2220.fit.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:146)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:387)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.joins

In [ ]:
lr = LogisticRegression(labelCol="", featuresCol="indexedFeatures",elasticNetParam=0.5)

In [ ]:
classifier = RandomForestClassifier(labelCol="flg_cmd_lowcostIndex", 
                                    featuresCol="indexedFeatures",
                                    maxDepth=15, numTrees=100)

model_rf = classifier.fit(data)